### Imports

In [52]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import statistics
from PIL import Image
from matplotlib import pyplot
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import adam_v2
from keras.utils.vis_utils import plot_model

### Pre-processing

In [53]:
IMAGE_SIZE = 256
IMAGE_CHANNELS = 3
PAINTING_TRAINING_SET_SIZE = 300  # we have a total of 300 monet paintings
PAINTINGS_DIR = '/kaggle/input/gan-getting-started/monet_jpg' 

real_paintings = []

for filename in os.listdir(PAINTINGS_DIR):
    path = os.path.join(PAINTINGS_DIR, filename)
    image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
    
    if len(real_paintings) < PAINTING_TRAINING_SET_SIZE:
        real_paintings.append(np.asarray(image))
    else: 
        real_paintings.append(np.asarray(image))
        
real_paintings = np.reshape(
    real_paintings, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
real_paintings = (real_paintings - 127.5) / 127.5

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  # Remove the CWD from sys.path while we load stuff.


### Config

In [54]:
NOISE_SIZE = 256
EPOCHS = 750
BATCH_SIZE = 29
DROPOUT_RATE = 0.3673

### Discriminator

In [55]:
def build_discriminator(image_shape): # image_shape should be 256x256x3
    model = Sequential(name='Discriminator')
    model.add(tfa.layers.SpectralNormalization((Conv2D(64, kernel_size=3, strides=1,
    input_shape=image_shape, padding='same'))))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(DROPOUT_RATE))
              
    model.add(tfa.layers.SpectralNormalization(Conv2D(64, kernel_size=4, strides=2, padding='same')))

    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(DROPOUT_RATE))
              
    model.add(tfa.layers.SpectralNormalization(Conv2D(128, kernel_size=3, strides=1, padding='same')))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(DROPOUT_RATE))

    model.add(tfa.layers.SpectralNormalization(Conv2D(128, kernel_size=4, strides=2, padding='same')))
    model.add(LeakyReLU(alpha=0.1))
              
    model.add(tfa.layers.SpectralNormalization(Conv2D(256, kernel_size=3, strides=1, padding='same')))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(DROPOUT_RATE))
              
    model.add(tfa.layers.SpectralNormalization(Conv2D(256, kernel_size=4, strides=2, padding='same')))
    model.add(LeakyReLU(alpha=0.1))
              
    model.add(tfa.layers.SpectralNormalization(Conv2D(512, kernel_size=3, strides=1, padding='same')))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(DROPOUT_RATE))
              
    model.add(Flatten())
    model.add(tfa.layers.SpectralNormalization(Dense(1, activation='sigmoid')))
              
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    
#     model.summary()
    plot_model(model, to_file='/kaggle/working/Deep_SN-GAN_discriminator.png', show_shapes=True, show_layer_names=True)
    
    return Model(input_image, validity, name='Discriminator')

### Generator

In [56]:
def build_generator(noise_size, channels):
    model = Sequential(name='Generator')
    
    model.add(Dense(4 * 4 * 512, activation='relu', input_dim=noise_size))
    model.add(Reshape((4, 4, 512)))
    
    model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Activation('relu'))
    
    model.add(tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Activation('relu'))
    
    model.add(tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Activation('relu'))    
    
    model.add(tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Activation('relu'))
    
    model.add(tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Activation('relu'))
    
    model.add(tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Activation('relu'))
        
#     model.summary()
    model.add(Conv2D(channels, kernel_size=3, strides=1, padding='same'))
    model.add(Activation('tanh'))
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    plot_model(model, to_file='/kaggle/working/Deep_SN-GAN_generator.png', show_shapes=True, show_layer_names=True)
    
    return Model(input, generated_image, name='Generator')


### Output Saving

In [57]:
def save_images(cnt, image, operation="train", tag="fake", path="training-output"): 
    out = None
    if tag == "fake":
        image = generator.predict(image)    
        out = image
    
    image = 0.5 * image + 0.5 
    image = (image[0] * 256).astype(np.uint8) # Save only first image in batch
        
    output_path = f'/kaggle/working/{path}'

    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    filename = os.path.join(output_path, f"{operation}-{cnt}-{tag}.png")
    im = Image.fromarray(image)
    im.save(filename)
    
    return out

### Plot

In [58]:
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.plot(d1_hist, label='d-real')
    pyplot.plot(d2_hist, label='d-fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    # plot discriminator accuracy
    pyplot.subplot(2, 1, 2)
    pyplot.plot(a1_hist, label='acc-real')
    pyplot.plot(a2_hist, label='acc-fake')
    pyplot.legend()
    # save plot to file
    
    output_path = '/kaggle/working/plots'

    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    pyplot.savefig(f'{output_path}/plot_line_plot_loss.png')
    pyplot.close()

### Train & Generate 

In [59]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = adam_v2.Adam(1.5e-4, 0.5)
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)
discriminator.trainable = False
validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE,1))
fixed_noise = np.random.normal(0, 1, (1, NOISE_SIZE))
cnt = 1

d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
for epoch in range(EPOCHS):
    idx = np.random.randint(0, real_paintings.shape[0], BATCH_SIZE)
    x_real = real_paintings[idx]  # x_real is 32 images of size 128*128*3

    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE)) # Noise is 32 samples of 128 pixels
    x_fake = generator.predict(noise)  # x_fake is 32 created images of 32x32x3    
    
    d_loss_real, discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
    d_loss_generated, discriminator_metric_generated = discriminator.train_on_batch(
    x_fake, y_fake)

    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
    generator_metric = combined.train_on_batch(noise,  np.ones((BATCH_SIZE, 1)))
    
    d1_hist.append(d_loss_real)
    d2_hist.append(d_loss_generated)
    g_hist.append(generator_metric)
    a1_hist.append(discriminator_metric_real)
    a2_hist.append(discriminator_metric_generated)
    plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)
    
    save_images(cnt, fixed_noise)
    cnt += 1

    print(f'Epoch: {epoch} | d_loss_real: {d_loss_real} | d_loss_fake: {d_loss_generated} | g_loss = {generator_metric} | d_acc_real: {discriminator_metric_real} | d_acc_fake: {discriminator_metric_generated}')

Epoch: 0 | d_loss_real: 0.681830644607544 | d_loss_fake: 0.699280321598053 | g_loss = [0.27072688937187195, 1.0] | d_acc_real: 0.7241379022598267 | d_acc_fake: 0.0
Epoch: 1 | d_loss_real: 0.382500559091568 | d_loss_fake: 1.139491081237793 | g_loss = [0.23414263129234314, 1.0] | d_acc_real: 1.0 | d_acc_fake: 0.0
Epoch: 2 | d_loss_real: 0.2777743637561798 | d_loss_fake: 1.4936952590942383 | g_loss = [0.5300784707069397, 1.0] | d_acc_real: 1.0 | d_acc_fake: 0.0
Epoch: 3 | d_loss_real: 0.49414733052253723 | d_loss_fake: 0.8119388818740845 | g_loss = [0.6408092379570007, 1.0] | d_acc_real: 1.0 | d_acc_fake: 0.0
Epoch: 4 | d_loss_real: 0.612670361995697 | d_loss_fake: 0.7510550022125244 | g_loss = [0.6619175672531128, 1.0] | d_acc_real: 1.0 | d_acc_fake: 0.0
Epoch: 5 | d_loss_real: 0.577846884727478 | d_loss_fake: 0.7537583708763123 | g_loss = [0.6681123971939087, 1.0] | d_acc_real: 1.0 | d_acc_fake: 0.0
Epoch: 6 | d_loss_real: 0.4435817003250122 | d_loss_fake: 0.7827392220497131 | g_loss = 

### SWD & Predictions

In [60]:
def sliced_wasserstein(A, B, dir_repeats=128, dirs_per_repeat=4):
    
    A = np.reshape(A, (A.shape[0], -1))
    B = np.reshape(B, (B.shape[0], -1))
    
    assert A.ndim == 2 and A.shape == B.shape                           # (neighborhood, descriptor_component)
    results = []
    for repeat in range(dir_repeats):
        dirs = np.random.randn(A.shape[1], dirs_per_repeat)             # (descriptor_component, direction)
        dirs /= np.sqrt(np.sum(np.square(dirs), axis=0, keepdims=True)) # normalize descriptor components for each direction
        dirs = dirs.astype(np.float32)
        projA = np.matmul(A, dirs)                                      # (neighborhood, direction)
        projB = np.matmul(B, dirs)
        projA = np.sort(projA, axis=0)                                  # sort neighborhood projections for each direction
        projB = np.sort(projB, axis=0)
        dists = np.abs(projA - projB)                                   # pointwise wasserstein distances
        results.append(np.mean(dists))                                  # average over neighborhoods and directions
    return np.mean(results)

SWD = []

# Noise to Paintings
for i in range(len(real_paintings)):
    ix = np.random.randint(0, real_paintings.shape[0], BATCH_SIZE)
    real = real_paintings[ix]
    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE)) # Noise is 32 samples of 128 pixels
                                   
    fake = save_images(i, noise, operation="test", tag="fake", path="test-output")
    save_images(i, real, operation="test", tag="real", path="test-output")
    
    SWD.append(sliced_wasserstein(real, fake))
    
    # Stop Early if needed
    print(i)
    if i == 15:
        break    
        

print(f"Mean SWD: {statistics.mean(SWD)}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Mean SWD: 0.2006738130613155


### Download results

In [61]:
os.chdir(r'/kaggle/working')

!tar -czf Out.tar.gz ./

from IPython.display import FileLink

FileLink(r'Out.tar.gz')

tar: ./Out.tar.gz: file changed as we read it


/kaggle/working/Out.tar.gz